# Improving Fine-tuned Model using RAG

Code authored by: Shaw Talebi <br>
Article link: https://towardsdatascience.com/how-to-improve-llms-with-rag-abdc132f76ac <br>
Video link: https://youtu.be/Ylz779Op9Pw?si=iOvBETQDrgoK_sO6 <br>
<br>
Colab: https://colab.research.google.com/drive/1peJukr-9E1zCo1iAalbgDPJmNMydvQms?usp=sharing

### imports

In [2]:
!pip3.10 install llama-index-embeddings-huggingface
!pip3.10 install llama-index


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3.10 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings, SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor

### Define Settings

In [4]:
# import any embedding model on HF hub (https://huggingface.co/spaces/mteb/leaderboard)
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
# Settings.embed_model = HuggingFaceEmbedding(model_name="thenlper/gte-large") # alternative model

Settings.llm = None
Settings.chunk_size = 256
Settings.chunk_overlap = 25

LLM is explicitly disabled. Using MockLLM.


### Read and Store Docs into Vector DB

In [6]:
    def load_documents_in_chunks(paths):
        return [SimpleDirectoryReader(path).load_data() for path in paths]

In [ ]:
# articles available here: {add GitHub repo}

import os

# List all files in the current directory
print(os.listdir('./'))


# pdf_path = "
# !pip install PyPDF2  # Install PyPDF2 if not already installed

# from PyPDF2 import PdfReader

# book_content = ""
# # Open and read the PDF
# reader = PdfReader(pdf_path)
# for page in reader.pages:
#     book_content += page.extract_text()

# print(book_content)

documents = SimpleDirectoryReader("Downloaded_Books").load_data()

['libextract.py', 'rag_example.ipynb', 'frontend.py', '.gitignore', '.env', '.venv', '.git', 'data', 'testAPI.ipynb', 'output.txt', 'Downloaded_Books']


Process SpawnPoolWorker-2:
Process SpawnPoolWorker-1:
Process SpawnPoolWorker-4:
Process SpawnPoolWorker-3:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/python@3.10/3.10.15/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/homebrew/Cellar/python@3.10/3.10.15/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/homebrew/Cellar/python@3.10/3.10.15/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/opt/homebrew/Cellar/python@3.10/3.10.15/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/queues.py", line 367, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'load_documents_in_chunks' on <module '__main__' (built-

KeyboardInterrupt: 

In [ ]:

print(len(documents))

In [ ]:
# store docs into vector DB
index = VectorStoreIndex.from_documents(documents)

### Set Up Search Function

In [ ]:
# set number of docs to retreive
top_k = 5

# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=top_k,
)

In [ ]:
# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.5)],
)

### Retrieve Relevant Docs

In [ ]:
# query documents
query = input("Enter your prompt:")
response = query_engine.query(query)

In [ ]:
# reformat response
context = "Context:\n"
for i in range(top_k):
    context = context + response.source_nodes[i].text + "\n\n"

print(context)

### Import LLM

In [ ]:
!pip3.10 install groq
!pip3.10 install python-dotenv

In [ ]:
from dotenv import load_dotenv

from groq import Groq

load_dotenv()

# Access the variables
api_key = os.getenv("GROQ_API_KEY")

client = Groq(
    api_key = api_key,
)

### Use LLM

In [ ]:
# prompt (with context)
prompt_template_w_context = lambda context, comment: f"""[INST]I am a student that is learning from a library

{context}
Please respond to the following comment. Use the context above if it is helpful.

{comment}
[/INST]
"""

In [ ]:
prompt = prompt_template_w_context(context, query)



chat_completion = client.chat.completions.create(
    messages=[
        {
            "role" : "user",
            "content" : prompt
        }
    ],
    model="llama3-8b-8192",
)

print(chat_completion.choices[0].message.content)